<center><h1>N2: Aprendizaje semi-supervisado</h1></center>

# N2: Métodos de aprendizaje semi-supervisado


# RECUERDA RELLENAR TUS DATOS A CONTINUACIÓN ANTES DE HACER NADA

In [1]:
# ===============================================================#
# Rellena AQUÍ tu nombre y apellidos antes de hacer nada
# ===============================================================#

NOMBRE = 'CarlosEsteban'
APELLIDOS = 'PosadaMejia'

# ===============================================================#
# NO MODIFIQUES ESTA PORCIÓN DE CÓDIGO, ES PARA LA EVALUACIÓN
# MUY IMPORTANTE: NO MODIFICAR
# ===============================================================#
from matplotlib.backends.backend_agg import FigureCanvasAgg
def encode_figure(fig):
    canvas = FigureCanvasAgg(fig)
    canvas.draw()
    img, (width, height) = canvas.print_to_buffer()
    return {'img': img, 'width': width, 'height': height}

answers = {}
answers['name'] = NOMBRE
answers['surname'] = APELLIDOS
answers['subject'] = '06MAIR_10_A_2020-21_ANS'
answers['ex'] = 'N2'
# ===============================================================#

En esta práctica aprenderemos a llevar a cabo un aprendizaje semi-supervisado. En concreto, estudiaremos el funcionamiento y la utilización del popular algoritmo de EM para aprender un modelo naive Bayes en semi-supervisado, así como la estrategia de aprendizaje conocida como "co-training".

Empezaremos por programar una implementación del clasificador que usaremos, el clásico Naive Bayes. Este clasificador probabilístico guarda los parámetros relativos a la distribución de probabilidad marginal de clase ($\{p(C=c)\}_{c=1}^{|C|}$) y las distribuciones de probabilidad condicional de las variables predictoras dada la clase ($\{p(X_i=x_i|C=c)\}_{x_i=1}^{|X_i|}$ para los diferentes valores de la variable clase $C=c$ y para todas las variables predictoras). Con estos parámetros se puede calcular la probabilidad de un caso, $\mathbf{x}$:
$$  p(C=c|\mathbf{x})\propto p(C=c)\prod_{i=1}^{v} p(x_i|C=c)$$
normalizándolos para todos los posibles valores de $c$ (todas las clases posibles) y 
con $v$ siendo el número de variables predictoras 
(2 en este caso) para que la 
suma sea $\sum_{c=1}^{|C|} p(C=c|\mathbf{x})=1$.

Por conveniencia, también guardaremos otras variables como la cardinalidad de las diferentes variables y el índice de la variable clase.


In [2]:
import numpy as np

class naiveBayes:
    def __init__(self, iClass, cardinalities):
        self.iClass = iClass
        self.cardinalities = cardinalities.copy()
        self.Pc = np.zeros(cardinalities[iClass])
        self.Pxc = []
        for i in np.arange(len(self.cardinalities)):
            aux = np.array([])
            if i != iClass:
                aux = np.zeros((cardinalities[i],cardinalities[iClass]))
            self.Pxc.append(aux)    

Una vez diseñado, vamos a lo importante. Un clasificador ha de 
poder ser aprendido y usado para predecir. Por ello, diseñaremos 
una función para cada procedimiento. 
Para predecir, se han de calcular las siguiente probabilidades:
$$ p(C=c)\prod_{i=1}^{v} p(x_i|C=c)$$
para todos los posibles valores de $c$ (todas las clases posibles) y 
con $v$ siendo el número de variables predictoras 
(2 en este caso). Finalmente, normalizamos para que la 
suma de $\sum_{c=1}^{|C|} p(C=c|x)=1$.
La función de predicción 
sería así:

In [3]:
def predictNB(model, instance):
    probs = model.Pc.copy()
    for i in np.arange(len(model.cardinalities)):
        if i != model.iClass:
            probs *= model.Pxc[i][instance[i],:]

    return probs/np.sum(probs)


Considerando la estructura definida para guardar los 
parámetros y lo visto en clase sobre aprendizaje de 
parámetros de máxima verosimilitud en un entorno de 
aprendizaje semi-supervisado, diseñaremos la función 
de aprendizaje:


In [4]:
def learnNB(L, U, pesosU, iClass, cardinalities, smoothing=1):
    modelo = naiveBayes(iClass, cardinalities)
    
    # Aprender de casos etiquetados
    for i in np.arange(L.shape[0]):
        # Actualizamos los parametros de la marginal P(c)
        modelo.Pc[L[i,iClass]] += 1
        
        for j in np.arange(len(cardinalities)):
            if j != iClass:
                # Actualizamos los parametros de las condicionales P(x_i|c)
                modelo.Pxc[j][L[i,j],L[i,iClass]] += 1

    # Aprender de casos no etiquetados
    for u in np.arange(U.shape[0]):
        # Actualizamos los parametros de la marginal de P(c)
        modelo.Pc += pesosU[u,:]
        
        for j in np.arange(len(cardinalities)):
            if j != iClass:
                # Actualizamos los parametros de las condicionales P(x_i|c)
                modelo.Pxc[j][U[u,j],:] += pesosU[u,:]
    
    modelo.Pc += smoothing # Laplace smoothing
    modelo.Pc /= np.sum(modelo.Pc)
    for j in np.arange(len(cardinalities)):
        if j != iClass:
            modelo.Pxc[j] += smoothing # Laplace smoothing
            modelo.Pxc[j] /= np.sum(modelo.Pxc[j],0)
    
    return modelo


Con esto completamos el diseño del clasificador Naive Bayes para el entorno semi-supervisado.

Ahora ya podemos crear el algoritmo EM, que itera los pasos E y M de forma sencilla hasta que converge.

**NOTA**: os recomiendo que primero os leáis todo el notebook y luego tratéis de completar el código que falta.


In [5]:
def EM(L, U, iClass, cardinalities, epsilon=0.001):
    modelo, pesosU = inicializar(L, U, iClass, cardinalities)

    convergencia = False
    it = 0
    while not convergencia:
        it += 1
        print('Iteracion', it)
        # PASO E
        pesosU = EStep(L, U, pesosU, modelo)

        antModelo = modelo
        
        # PASO M
        modelo = MStep(L, U, pesosU, iClass, cardinalities, antModelo)## Tu código aquí ##
    
        # Comprobacion de convergencia
        convergencia = testConvergencia(modelo, antModelo, epsilon)
    
    return modelo, pesosU

Faltarían por programar las tres funciones fundamentales 
del EM: el paso E (<b>EStep</b>),  el paso M (<b>MStep</b>) 
y la función que comprueba la convergencia (<b>testConvergencia</b>). 
Además, falta la inicialización donde asignamos valores iniciales 
a los pesos de las instancias no etiquetadas para aprender la 
primera versión del modelo. En concreto, en esta ocasión 
asignaremos a todos los casos la misma probabilidad de 
pertenecer a cualquier clase:


In [6]:
def inicializar(L, U, iClass, cardinalities):
    # Todos los elementos en U tienen la misma probabilidad de pertenecer a cualquier clase
    pesosU = np.ones((U.shape[0],cardinalities[iClass]))
    pesosU /= cardinalities[iClass]
    
    modelo = learnNB(L, U, pesosU, iClass, cardinalities)

    return modelo, pesosU


El paso E consiste en recalcular los pesos dado un modelo y el paso M en aprender una nueva versión del modelo:
    

In [7]:
def EStep(L, U, pesosU, modelo):
    nPesosU = np.zeros(pesosU.shape)
    for u in np.arange(U.shape[0]):
        nPesosU[u,:] = predictNB(modelo, U[u,:])

    return nPesosU

def MStep(L, U, pesosU, iClass, cardinalities, antModelo):
    modelo = learnNB(L, U, pesosU, iClass, cardinalities)
    return modelo


Mediremos la convergencia atendiendo al criterio de si 
la distancia euclídea entre los parámetros del paso anterior 
y del actual es menor que <i>epsilon</i> o no:


In [8]:
def testConvergencia(modeloA, modeloB, epsilon=0.001):
    resultado = np.sum((modeloA.Pc-modeloB.Pc)**2)

    for j in np.arange(len(modeloA.cardinalities)):
        if j != modeloA.iClass:
            resultado += np.sum((modeloA.Pxc[j]-modeloB.Pxc[j])**2)
  
    resultado = np.sqrt(resultado)
    
    return resultado < epsilon


Una vez hemos diseñado completamente el algoritmo EM y el clasificador NB, podemos proceder a su uso. 
Creamos un dataset y hacemos la llamada al EM:


In [9]:
from sklearn.datasets import make_classification

np.random.seed(23)
nPredVars = 10
iClass = nPredVars
nSampleXsubset = 20
cardinalities = np.repeat(2,nPredVars+1)

# Simulamos un dataset
X,y = make_classification(n_samples=nSampleXsubset*3,n_features=nPredVars,n_redundant=0)
X[X<0] = 0;X[X>0] = 1 # discretizamos las variables predictoras
X = X.astype(int)
y.shape=(len(y),1)

L = np.concatenate((X[:nSampleXsubset,:],
                    y[:nSampleXsubset,:]),axis=1)
U = np.concatenate((X[nSampleXsubset:(nSampleXsubset*2),:],
                    y[nSampleXsubset:(nSampleXsubset*2),:]),axis=1)
U[:,iClass] = np.nan # eliminamos la etiqueta de un subconjunto de elementos

modelo, pesos = EM(L, U, iClass, cardinalities, 0.001)

# INICIO - NO TOCAR : necesario para la evaluación del notebook
answers['1'] = {'modelo': modelo,
                'pesos': pesos}
# FIN - NO TOCAR : necesario para la evaluación del notebook

Iteracion 1
Iteracion 2
Iteracion 3
Iteracion 4
Iteracion 5
Iteracion 6
Iteracion 7
Iteracion 8
Iteracion 9
Iteracion 10
Iteracion 11
Iteracion 12



Como en anteriores ocasiones, podemos estudiar la bondad del agrupamiento ya que se conoce la realidad:


In [10]:
test = np.concatenate((X[nSampleXsubset*2:,:],
                    y[nSampleXsubset*2:,:]),axis=1)

realLabels = test[:,iClass]
predLabels = np.zeros(realLabels.shape)
for i in np.arange(test.shape[0]):
    probs = predictNB(modelo, test[i,:])
    predLabels[i] = np.argmax(probs)

In [11]:
def matriz_confusion(cat_real, cat_pred, nClasses):
    mat = np.array([[np.sum(np.logical_and(cat_real==i, cat_pred==j)) 
                     for j in np.arange(nClasses)] 
                    for i in np.arange(nClasses)])
    return(mat)

def medida_error(mat):
    tot = np.sum(mat)
    aux = mat.copy()
    np.fill_diagonal(aux, 0)
    return float(np.sum(aux))/tot

def medida_precision(mat, l, k):
    return mat[l,k]/float(np.sum(mat[:,k]))

def medida_recall(mat, l, k):
    return mat[l,k]/float(np.sum(mat[l,:]))

def medida_f1(mat):
    prec = medida_precision(mat, 1, 1)
    rec = medida_recall(mat, 1, 1)
    if (prec+rec)==0:
        return 0
    else:
        return 2*prec*rec/(prec+rec)

mC = matriz_confusion(realLabels,predLabels,cardinalities[iClass])

print(mC)
print('El error del clasificador es = ', medida_error(mC))
print('El valor F1 del clasificador es = ', medida_f1(mC))

# INICIO - NO TOCAR : necesario para la evaluación del notebook
answers['1'] = {'mC': mC}
# FIN - NO TOCAR : necesario para la evaluación del notebook

[[9 2]
 [3 6]]
El error del clasificador es =  0.25
El valor F1 del clasificador es =  0.7058823529411765


<hr>
<h2>Implementaciones en librerías de Python</h2>

La librería ScikitLearn implementa diversas versiones del clasificador NB. En el caso de <b>MultinomialNB</b>, la distribución de probabilidad que se modela es diferente: una distribución de probabilidad multinomial donde el valor de cada variable predictora es el conteo de esta distribución de probabilidad:

\begin{aligned}
\Pr(X_{1}=x_{1},\dots, X_{v}=x_{v})&{}={\begin{cases}{\displaystyle {n! \over x_{1}!\cdot\dots\cdot x_{k}!}p_{1}^{x_{1}}\cdot \dots \cdot p_{v}^{x_{v}}},\quad &{\text{when }}\sum _{i=1}^{k}x_{i}=n\\\\0&{\text{otherwise,}}\end{cases}}
\end{aligned}
para cualquier conjunto $\{x_1, \dots, x_v\}$ de valores no negativos.

Si quisiésemos incluir esta versión del NB en el algoritmo EM, deberíamos modificar las funciones específicas que hacen llamadas al modelo pero no la general iterativa del EM.

En primer lugar, deberíamos preparar una función para el aprendizaje del modelo cuando los datos son semi-supervisados:


In [12]:
from sklearn.naive_bayes import MultinomialNB

def sklearnNB(L, U, pesosU, iClass, cardinalities):
    modelo = MultinomialNB()

    modelo.partial_fit(np.delete(L, iClass, axis=1),
                       L[:,iClass],
                       classes=[0,1])

    for c in np.arange(cardinalities[iClass]):
        impU = U.copy()
        impU[:,iClass] = c
        modelo.partial_fit(np.delete(impU, iClass, axis=1),
                           impU[:,iClass],
                           sample_weight = pesosU[:,c])

    return modelo


La función de predicción del NB, en este caso, ya está integrada en la definición de la clase de Scikit-learn.

Las cuatro funciones auxiliares del EM se deben adecuar a este nuevo modelo: 


In [13]:
def inicializar(L, U, iClass, cardinalities):
    pesosU = np.ones((U.shape[0],cardinalities[iClass]))
    pesosU /= cardinalities[iClass]
    
    modelo = sklearnNB(L, U, pesosU, iClass, cardinalities)

    return modelo, pesosU

def EStep(L, U, pesosU, modelo):

    nPesosU = modelo.predict_proba(np.delete(U, iClass, axis=1))

    return nPesosU

def MStep(L, U, pesosU, iClass, cardinalities, antModelo):
    modelo = sklearnNB(L, U, pesosU, iClass, cardinalities)
    return modelo

def testConvergencia(modeloA, modeloB, epsilon=0.001):
    resultado = np.sum((np.exp(modeloA.class_log_prior_) - 
                        np.exp(modeloB.class_log_prior_))**2)
    resultado += np.sum((np.exp(modeloA.feature_log_prob_) - 
                         np.exp(modeloB.feature_log_prob_))**2)

    resultado = np.sqrt(resultado)
    
    return resultado < epsilon


Una vez hemos diseñado completamente las distintas funciones del algoritmo EM y esta nueva versión del NB, podemos proceder a su uso. 
Creamos un dataset y hacemos la llamada al EM:


In [14]:
np.random.seed(23)
nPredVars = 10
iClass = nPredVars
nSampleXsubset = 20
cardinalities = np.repeat(2,nPredVars+1)

# Simulamos un dataset
X,y = make_classification(n_samples=nSampleXsubset*3,n_features=nPredVars,n_redundant=0)
X[X<0] = 0;X[X>0] = 1 # discretizamos las variables predictoras
X = X.astype(int)
y.shape=(len(y),1)

L = np.concatenate((X[:nSampleXsubset,:],
                    y[:nSampleXsubset,:]),axis=1)
U = np.concatenate((X[nSampleXsubset:(nSampleXsubset*2),:],
                    y[nSampleXsubset:(nSampleXsubset*2),:]),axis=1)
U[:,iClass] = np.nan

skmodelo, skpesos = EM(L, U, iClass, cardinalities, 0.001)
print(skpesos)

Iteracion 1
Iteracion 2
Iteracion 3
Iteracion 4
Iteracion 5
Iteracion 6
[[0.37452499 0.62547501]
 [0.20208038 0.79791962]
 [0.4381833  0.5618167 ]
 [0.61770176 0.38229824]
 [0.53587632 0.46412368]
 [0.52671695 0.47328305]
 [0.46247666 0.53752334]
 [0.29304468 0.70695532]
 [0.48819531 0.51180469]
 [0.34138085 0.65861915]
 [0.56828363 0.43171637]
 [0.22954694 0.77045306]
 [0.63735359 0.36264641]
 [0.42909102 0.57090898]
 [0.21850953 0.78149047]
 [0.52680548 0.47319452]
 [0.66058945 0.33941055]
 [0.460702   0.539298  ]
 [0.52071844 0.47928156]
 [0.6150956  0.3849044 ]]


In [15]:
test = np.concatenate((X[nSampleXsubset*2:,:],
                       y[nSampleXsubset*2:,:]),axis=1)

skRealLabels = test[:,iClass]
skPredLabels = np.zeros(realLabels.shape)
test = np.delete(test,iClass,1)
skPredLabels = np.argmax(skmodelo.predict_proba(test), axis=1)

In [16]:
mC = matriz_confusion(skRealLabels,skPredLabels,cardinalities[iClass])

print(mC)
print('El error del clasificador es = ', medida_error(mC))
print('El valor F1 del clasificador es = ', medida_f1(mC))

[[7 4]
 [2 7]]
El error del clasificador es =  0.3
El valor F1 del clasificador es =  0.7000000000000001



El algoritmo EM, ejecutado en una única ocasión, puede dar un resultado no óptimo: se está eligiendo una inicialización que lleva al algoritmo a encallar en un óptimo local. Si se ejecuta en varias ocaciones, eventualmente se obtendrá el resultado óptimo.

Podríamos comparar el resultado de nuestro algoritmo y el de la implementación de ScikitLearn para observar si devuelven el mismo resultado:
    

In [17]:
# Si comparamos el resultado de ambos algoritmos, el nuestro y el de ScikitLearn
mC_comp = matriz_confusion(predLabels,skPredLabels,cardinalities[iClass])

print('Matriz de confusión:')
print(mC_comp)
print('El valor del error (diferencia) es = ', medida_error(mC_comp))
print('El valor F1 del clasificador es = ', medida_f1(mC_comp))

Matriz de confusión:
[[9 3]
 [0 8]]
El valor del error (diferencia) es =  0.15
El valor F1 del clasificador es =  0.8421052631578948


In [19]:
# PREGUNTAS:

# 1. Al utilizar Naive Bayes estamos realizando varias asunciones, una de ellas, muy importante, relacionada con nuestras variables. ¿Cuál es esta asunción?

# a) Que nuestras variables esten escaladas en el mismo rango
# b) Que nuestras variables son independientes unas de otras
# c) Que nuestras variables no contengan valores nulos

respuesta1 = 'b'

# 2. ¿Qué ventaja hay en usar EM NB respecto a Naive Bayes?

# a) El proceso de aprendizaje es más rápido gracias al algoritmo de EM
# b) El algoritmo de EM permite poder entrenar un modelo aún cuando no disponemos de todas las etiquetas
# c) Permite relajar la condición de la independencia de las variables

respuesta2 = 'b'

# INICIO - NO TOCAR : necesario para la evaluación del notebook
answers['2'] = {'respuesta': respuesta1}
answers['3'] = {'respuesta': respuesta2}
# FIN - NO TOCAR : necesario para la evaluación del notebook

# RECUERDA EJECUTAR ESTA CELDA ÚNICAMENTE CUANDO HAYAS FINALIZADO EL NOTEBOOK

In [20]:
################################################################################
# ATENCIÓN, MUY IMPORTANTE: EJECUTA ESTA CELDA SIN MODIFICAR NADA!
#
# ESTA CELDA SE ENCARGA DE COMPRIMIR LOS RESULTADOS OBTENIDOS Y MANDARLOS
# AL SERVIDOR PARA SU VERIFICACIÓN Y ALMACENAJE.
#
# SI NO LA EJECUTAS, ¡¡¡NO PODRÁS SER EVALUADX!!!
################################################################################

import zmq
import base64
import pickle
import bz2
import sys

def send_results(answers):

    def compress_data(data):
        data_pkl = pickle.dumps(data)
        data_bz2 = bz2.compress(data_pkl)
        return data_bz2

    if NOMBRE == 'TuNombre' or APELLIDOS == 'TusApellidos' or \
    NOMBRE.strip() == '' or APELLIDOS.strip() == '':
        print('Rellena tu nombre y apellidos en la primera celda del notebook!')
    else:
        print('Conectando con el servidor de evaluación...', end='\t')
        context = zmq.Context()
        socket = context.socket(zmq.REQ)
        socket.connect(base64.b64decode(b'dGNwOi8vMTU4LjQyLjE3MC4xMzU6MzM4OQ==').decode('ascii'))
        print('OK')

        # Compressing data
        print('Comprimiendo las respuestas...', end='\t')
        data = compress_data(answers)
        print('OK')

        print('Enviando datos...', end='\t')
        socket.send(data)
        message = socket.recv()
        if message == data:
            print(f'OK')
            print('Entrega realizada con éxito! :)')
        else:
            print('ERROR! Por favor vuelve a ejecutar la celda. Si los problemas persisten ponte en contacto con el profesor.')    


send_results(answers)

Conectando con el servidor de evaluación...	OK
Comprimiendo las respuestas...	OK
Enviando datos...	OK
Entrega realizada con éxito! :)
